In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import xarray as xr
import numpy as np

In [16]:
data = xr.DataArray(np.random.randn(10), dims=("x"), coords={"x": np.arange(10)})

In [20]:
data.values

array([-0.73675757, -0.64201974, -0.70003928,  0.59195098,  0.02750936,
       -0.24321775, -0.07676518, -0.42603707,  0.20764745, -0.36070899])